In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.sparse import random
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier

In [2]:
steam_row = pd.read_csv(r"C:\Users\82103\sih\web_crawling\SIH\sih_eda\sih_processing.csv", sep = ',')

In [3]:
steam_row.head()

,Unnamed: 0,appid,24_Hour_Peak,All_time_peak,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,...,Sexual Content,Utilities,Audio Production,Sports,360 V,Massively Multiplayer,Action,Education,Casual,Violent
0,0,294100,20930,60742,12492,1401,5368,921,34.99,34.99,...,0,0,0,0,0,0,0,0,0,0
1,1,960090,13428,53891,4018,277,1532,62,13.99,13.99,...,0,0,0,0,0,0,0,0,0,0
2,2,504230,1234,4148,1316,310,370,582,19.99,19.99,...,0,0,0,0,0,0,1,0,0,0
3,3,588650,4553,14880,1841,183,868,213,14.99,24.99,...,0,0,0,0,0,0,1,0,0,0
4,4,264710,4205,50876,1943,313,941,277,29.99,29.99,...,0,0,0,0,0,0,0,0,0,0


In [4]:
y = steam_row.loc[:, "Review"]
X = steam_row.drop("Review", axis=1)
X.head()

,Unnamed: 0,appid,24_Hour_Peak,All_time_peak,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,...,Sexual Content,Utilities,Audio Production,Sports,360 V,Massively Multiplayer,Action,Education,Casual,Violent
0,0,294100,20930,60742,12492,1401,5368,921,34.99,34.99,...,0,0,0,0,0,0,0,0,0,0
1,1,960090,13428,53891,4018,277,1532,62,13.99,13.99,...,0,0,0,0,0,0,0,0,0,0
2,2,504230,1234,4148,1316,310,370,582,19.99,19.99,...,0,0,0,0,0,0,1,0,0,0
3,3,588650,4553,14880,1841,183,868,213,14.99,24.99,...,0,0,0,0,0,0,1,0,0,0
4,4,264710,4205,50876,1943,313,941,277,29.99,29.99,...,0,0,0,0,0,0,0,0,0,0


In [5]:

from scipy.sparse import random
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size = 0.2,
    random_state = 1234, 
    stratify = y 
)

In [6]:
y_train.value_counts(normalize=True)

Positive       0.419854
No_interest    0.412854
Mixed          0.142158
Negative       0.025134
Name: Review, dtype: float64

In [7]:
y_val.value_counts(normalize=True)

Positive       0.419847
No_interest    0.412928
Mixed          0.142147
Negative       0.025078
Name: Review, dtype: float64

In [8]:
from sklearn.model_selection import StratifiedKFold
str_kfold = StratifiedKFold(n_splits=5, random_state=1234, shuffle=True )

In [9]:
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
X_train_numeric = X_train.drop(categorical_cols, axis=1)
X_train = X_train.drop('new_date', axis=1)
import re

# 특수 문자 패턴 설정
special_chars_pattern = r"[^\w\s]"

# 특징 이름 수정
new_feature_names = []
for feature_name in X_train.columns:
    new_feature_name = re.sub(special_chars_pattern, "", feature_name)
    new_feature_names.append(new_feature_name)

# 수정된 특징 이름으로 열 이름 변경
X_train.columns = new_feature_names

In [10]:
lgbm = LGBMClassifier( random_state= 1234, n_jobs=-1)

parameters = {
    "n_estimators" : [ 10, 30, 50, 100, 300, 500],
    "learning_rate" : [0.01, 0.1, 0.2, 0.3, 0.5],

    "max_depth" : [ 2,3,4,5,6,10],     # 인위적인 높이 조절
    "min_split_gain" : [0, 0.1, 0.2, 0.3, 0.4], # 평가지표 상 분화 조건값 
    "subsample" : [0.3, 0.4, 0.5, 0.6, 0.9 ], # 학습용 데이터의 수를 얼마나 샘플링할지
    "colsample_bytree" : [0.3,0.4,0.5,0.6, 0.9],

    "reg_alpha": [0, 0.01, 0.1, 0.3],
    "reg_lambda" : [0, 0.01, 0.1, 0.3]
}
n_iter = 50
lgbm_kfold_rgs = RandomizedSearchCV(  lgbm,
                                   param_distributions=parameters,
                                   cv = str_kfold,
                                   scoring="accuracy",
                                   n_jobs= -1,
                                   random_state=1234,
                                   n_iter= n_iter
)
lgbm_kfold_rgs.fit(X_train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1234, shuffle=True),
                   estimator=LGBMClassifier(random_state=1234), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5, 0.6,
                                                             0.9],
                                        'learning_rate': [0.01, 0.1, 0.2, 0.3,
                                                          0.5],
                                        'max_depth': [2, 3, 4, 5, 6, 10],
                                        'min_split_gain': [0, 0.1, 0.2, 0.3,
                                                           0.4],
                                        'n_estimators': [10, 30, 50, 100, 300,
                                                         500],
                                        'reg_alpha': [0, 0.01, 0.1, 0.3],
                                        'reg_lambda': [0, 0.01, 0.1, 0.3],
             

In [13]:
from lightgbm import LGBMClassifier
# baseline
lgbm = LGBMClassifier( random_state= 1234, n_jobs=-1)
scores = cross_val_score( lgbm, 
                         X_train, y_train,
                         cv = str_kfold, # --> 고정적으로 CV할 때 데이터셋 고정..
                         scoring = "accuracy")

for iter_count, acc in enumerate( scores):
    print("LGBM {0}번째 검증 ACC:{1:.4f}".format(iter_count+1,acc ))
print("LGBM 모델 Accuracy 평균:", scores.mean())
print("LGBM 모델 Accuracy std:", scores.std())

LGBM 1번째 검증 ACC:1.0000
LGBM 2번째 검증 ACC:1.0000
LGBM 3번째 검증 ACC:1.0000
LGBM 4번째 검증 ACC:1.0000
LGBM 5번째 검증 ACC:1.0000
LGBM 모델 Accuracy 평균: 1.0
LGBM 모델 Accuracy std: 0.0
